# Amazon Web Scraper

**Requirements:**

* Selenium
* BeautifulSoup

In [ ]:
# pip install bs4
import csv
from bs4 import BeautifulSoup

In [ ]:
# Firefox & Chrome
# pip install selenium
from selenium import webdriver

In [ ]:
# Microsoft Edge
# pip install msedge-selenium-tools
#from msedge.selenium_tools import Edge, EdgeOpti

In [ ]:
driver = webdriver.Chrome(executable_path=':c\anaconda3\Lib\site-packages\selenium\webdriver\chrome')

# Startup The Webdriver

In [ ]:
# Firefox & Chrome
# https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-win64.zip
# https://sites.google.com/a/chromium.org/chromedriver/
#driver = webdriver.Firefox()
#driver = webdriver.Chrome()

In [ ]:
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
#options = EdgeOptions()
#options.use_chromium =True
#driver = Edge(options=options)

In [ ]:
url = 'https://www.amazon.com'
driver.get(url)

In [ ]:
# https://www.amazon.com/s?k=ultrawide+monitor
# Generate URL From Search Term
def get_url(search_term):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [ ]:
url =get_url('apple iphones')
print(url)
driver.get(url)

# Extract The Collection

In [ ]:
soup =BeautifulSoup(driver.page_source, 'html.parser')
print(soup.title.text)

In [ ]:
# //div[@data-component-type='s-search-result']
results = soup.find_all('div', {'data-component-type': 's-search-result'})
print(len(results))

# Prototype The Record

In [ ]:
item = results[0]
atag = item.h2.a

In [ ]:
description = atag.text.strip()

In [ ]:
url = 'https://www.amazon.in' + atag.get('href')

In [ ]:
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

In [ ]:
rating = item.i.text

In [ ]:
review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text

In [ ]:
image = item.find('img', {'class': 's-image'}).get('src')

# Generalize The Pattern

In [ ]:
# Extract And Return Data From Record
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    try:
        price = item.find('span', 'a-price').find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    image = item.find('img', {'class': 's-image'}).get('src')
    result = (description, price, rating, review_count, url, image)
    return result

In [ ]:
item = results[2]
extract_record(item)

In [ ]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [ ]:
records[0]

In [ ]:
for row in records:
    print(row[1])

# Getting The Next Page

In [ ]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}'
    search_term = search_term.replace(' ', '+')
    
    # Add Term Query To URL
    url = template.format(search_term)
    
    # Add Page Query Placeholder
    url += '&page={}'
    
    return url

# Putting It All Together

In [ ]:
import csv
from bs4 import BeautifulSoup
driver = webdriver.Chrome(executable_path='/users/sairam/Downloads/chromedriver')

def get_url(search_term):
    template = 'https://www.amazon.in/s?k={}'
    search_term = search_term.replace(' ', '+')
    
    # Add Term Query To URL
    url = template.format(search_term)
    
    # Add Page Query Placeholder
    url += '&page={}'
    
    return url

def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    try:
        price = item.find('span', 'a-price').find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    image = item.find('img', {'class': 's-image'}).get('src')   
    result = (description, price, rating, review_count, url, image)
    return result

'''Run Main Program Routine'''
def main(search_term):
    # Startup The Webdriver
   # driver.get(url)
    
    records = []
    url =get_url(search_term)
    
    for page in range(1, 3):
        driver.get(url.format(page))
        soup =BeautifulSoup(driver.page_source, 'html.parser')
        
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    # Save Results To CSV File
    with open('Results123.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Reviews Count', 'URL', 'Image Link'])
        writer.writerows(records)

In [ ]:
main('apple iphones')